In [ ]:
"""
L1 Density Analysis Around Ligation Points
Analysis of L1 element distribution patterns around chromatin ligation points in human genome (hg38)
"""

import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt
import time
from datetime import timedelta
import os

# Set working directory and create output directory for plots
work_dir = "your_working_directory"  # Set your working directory path
out_dir = os.path.join(work_dir, "out")
os.makedirs(out_dir, exist_ok=True)
os.chdir(work_dir)

# Load data
print("Loading L1 elements...")
te_df = pd.read_csv('hg38_te_filtered.bed', sep='\t')
l1_df = te_df[te_df['repFamily'] == 'L1'].copy()
print(f"Loaded {len(l1_df)} L1 elements")

print("Loading LP datasets...")
lp_df1 = pd.read_csv('LP_dataset_1_second_method.csv')
lp_df2 = pd.read_csv('LP_dataset_2_second_method.csv')
print(f"Loaded {len(lp_df1)} LPs from dataset 1")
print(f"Loaded {len(lp_df2)} LPs from dataset 2")

def analyze_l1_density(chrom, point, window=50000, bins=200):
    """
    Analyze L1 density around a given genomic point
    
    Parameters:
    chrom (str): Chromosome name
    point (int): Genomic position
    window (int): Window size around point (default 50kb)
    bins (int): Number of bins for density calculation
    
    Returns:
    tuple: (bin_centers, plus_strand_histogram, minus_strand_histogram)
    """
    window_start = max(0, point - window)
    window_end = point + window
    
    mask = (l1_df['genoName'] == chrom) & \
           (l1_df['genoEnd'] >= window_start) & \
           (l1_df['genoStart'] <= window_end)
    
    window_l1s = l1_df[mask].copy()
    window_l1s['distance'] = ((window_l1s['genoStart'] + window_l1s['genoEnd'])/2 - point)
    
    plus_strand = window_l1s[window_l1s['strand'] == '+']['distance']
    minus_strand = window_l1s[window_l1s['strand'] == '-']['distance']
    
    bin_edges = np.linspace(-window, window, bins+1)
    plus_hist, _ = np.histogram(plus_strand, bins=bin_edges)
    minus_hist, _ = np.histogram(minus_strand, bins=bin_edges)
    
    return bin_edges[:-1], plus_hist, minus_hist

def analyze_lps_in_batches(lp_df, dataset_name, batch_size=200, sigma=1):
    """
    Analyze L1 density around LPs in batches
    
    Parameters:
    lp_df (DataFrame): DataFrame with LP coordinates
    dataset_name (str): Name for output files
    batch_size (int): Number of LPs to process in each batch
    sigma (float): Smoothing parameter
    """
    bins = 200
    total_plus_lp1 = np.zeros(bins)
    total_minus_lp1 = np.zeros(bins)
    total_plus_lp2 = np.zeros(bins)
    total_minus_lp2 = np.zeros(bins)
    count = 0
    
    start_time = time.time()
    total_lps = len(lp_df)
    
    for batch_start in range(0, total_lps, batch_size):
        batch_end = min(batch_start + batch_size, total_lps)
        batch_df = lp_df.iloc[batch_start:batch_end]
        
        for idx, row in batch_df.iterrows():
            try:
                # Process LP1
                bin_centers, plus_hist, minus_hist = analyze_l1_density(row['chrom'], row['LP1'])
                total_plus_lp1 += plus_hist
                total_minus_lp1 += minus_hist
                
                # Process LP2
                _, plus_hist, minus_hist = analyze_l1_density(row['chrom'], row['LP2'])
                total_plus_lp2 += plus_hist
                total_minus_lp2 += minus_hist
                
                count += 1
                
            except Exception as e:
                print(f"Error processing row {idx}: {e}")
        
        # Calculate current averages and save plot
        avg_plus_lp1 = gaussian_filter1d(total_plus_lp1 / count, sigma=sigma)
        avg_plus_lp2 = gaussian_filter1d(total_plus_lp2 / count, sigma=sigma)
        avg_minus_lp1 = gaussian_filter1d(total_minus_lp1 / count, sigma=sigma)
        avg_minus_lp2 = gaussian_filter1d(total_minus_lp2 / count, sigma=sigma)
        
        # Progress update
        elapsed_time = time.time() - start_time
        print(f"\nProcessed {count}/{total_lps} LPs ({(count/total_lps*100):.1f}%)")
        print(f"Time elapsed: {timedelta(seconds=int(elapsed_time))}")
        
        # Save current plot
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
        
        ax1.plot(bin_centers, avg_plus_lp1, label='LP1', color='blue')
        ax1.plot(bin_centers, avg_plus_lp2, label='LP2', color='lightblue', linestyle='--')
        ax1.axvline(x=0, color='black', linestyle='--', alpha=0.5)
        ax1.set_title(f'L1 Plus Strand Density ({dataset_name}, {count} LPs)')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        ax2.plot(bin_centers, avg_minus_lp1, label='LP1', color='red')
        ax2.plot(bin_centers, avg_minus_lp2, label='LP2', color='pink', linestyle='--')
        ax2.axvline(x=0, color='black', linestyle='--', alpha=0.5)
        ax2.set_title('L1 Minus Strand Density')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(os.path.join(out_dir, f'{dataset_name}_batch_{count}.png'))
        plt.close()

# Run analysis
print("\nAnalyzing Dataset 1...")
result1 = analyze_lps_in_batches(lp_df1, "Dataset1")

print("\nAnalyzing Dataset 2...")
result2 = analyze_lps_in_batches(lp_df2, "Dataset2")